In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

<h3>Connecting to OpenAI</h3>The next cell loads environment variables from the .env file and connects to OpenAI.I

In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')


# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

<h3>Call to a Frontier model</h3>

In [4]:
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! It’s great to hear from you. How can I assist you today?


In [5]:
# headers when fetching websites:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
ed = Website("https://simple.wikipedia.org/wiki/Quantum_computer")
print(ed.title)
print(ed.text)

Quantum computer - Simple English Wikipedia, the free encyclopedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Getting around
Main page
Simple start
Simple talk
New changes
Show any page
Help
Contact us
About Wikipedia
Search
Search
Appearance
Give to Wikipedia
Create account
Log in
Personal tools
Give to Wikipedia
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
Beginning
1
References
2
Other websites
Toggle the table of contents
Quantum computer
60 languages
Afrikaans
Alemannisch
العربية
Azərbaycanca
বাংলা
Български
Bosanski
Català
Чӑвашла
Čeština
Dansk
Deutsch
Eesti
Ελληνικά
English
Español
Esperanto
Euskara
Français
Frysk
Gaeilge
한국어
Հայերեն
हिन्दी
Hrvatski
Bahasa Indonesia
Italiano
עברית
ಕನ್ನಡ
ქართული
Latviešu
Lietuvių
Magyar
മലയാളം
Malti
Bahasa Melayu
Nederlands
日本語
Norsk bokmål
ଓଡ଼ିଆ
Oʻzbekcha / ўзбекча
ਪੰਜਾਬੀ
Polski
Português
Română
Русский
Shqip
Slovenčina
Slovenščina
Српски / srpski
Srpskohrvatski / ср

In [7]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled called ----- {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled called ----- Quantum computer - Simple English Wikipedia, the free encyclopedia
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Jump to content
Main menu
Main menu
move to sidebar
hide
Getting around
Main page
Simple start
Simple talk
New changes
Show any page
Help
Contact us
About Wikipedia
Search
Search
Appearance
Give to Wikipedia
Create account
Log in
Personal tools
Give to Wikipedia
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
Beginning
1
References
2
Other websites
Toggle the table of contents
Quantum computer
60 languages
Afrikaans
Alemannisch
العربية
Azərbaycanca
বাংলা
Български
Bosanski
Català
Чӑвашла
Čeština
Dansk
Deutsch
Eesti
Ελληνικά
English
Español
Esperanto
Euskara
Français
Frysk
Gaeilge
한국어
Հայերեն
हिन्दी
Hrvatski
Bahasa Indonesia
Italiano
עברית
ಕನ್ನಡ

In [10]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [11]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled called ----- Quantum computer - Simple English Wikipedia, the free encyclopedia\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nJump to content\nMain menu\nMain menu\nmove to sidebar\nhide\nGetting around\nMain page\nSimple start\nSimple talk\nNew changes\nShow any page\nHelp\nContact us\nAbout Wikipedia\nSearch\nSearch\nAppearance\nGive to Wikipedia\nCreate account\nLog in\nPersonal tools\nGive to Wikipedia\nCreate account\nLog in\nPages for logged out editors\nlearn more\nContributions\nTalk\nContents\nmove to sidebar\nhide\nBeginning\n1\nReferences\n2\nOther websites\nToggle the table of contents\nQua

In [12]:

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [13]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://simple.wikipedia.org/wiki/Quantum_computer")

# Summary of "Quantum Computer - Simple English Wikipedia"

The Simple English Wikipedia page on **quantum computers** discusses the basic principles of quantum computing, which involve the use of quantum mechanics concepts such as **superposition** and **entanglement**. Here are the key points from the article:

- **Definition**: A quantum computer is a model for building computers that leverage quantum properties to perform operations on data.
- **Qubits**: Unlike classical computers that use bits (which are either on or off), quantum computers utilize **qubits**, which can represent both states simultaneously due to superposition.
- **Rapid Progress**: Quantum computing is a relatively new field, with ongoing research supported by governments and military funding for potential applications like cryptanalysis.
- **Comparison**: While quantum computers cannot compute functions that classical computers cannot, they can solve certain problems much faster, highlighting their potential advantage.
- **Future Prospects**: The text emphasizes the possibility of building large-scale quantum computers that could outperform classical computers in specific tasks.

Overall, the article provides an accessible overview of the fundamental concepts of quantum computing, its current state, and its potential future implications.